# Units in Python

### [Astropy Units](http://docs.astropy.org/en/stable/units/index.html#module-astropy.units.si)

In [1]:
import numpy as np
import pandas as pd

from astropy import units as u
from astropy import constants as const
from astropy.units import imperial
imperial.enable()

#### *Note: because we imported the `units` package as `u`, you cannot use **u** as a variable name.*

---

### You can use the units by themselves (`u.`):

In [2]:
u.m    # The unit of meters

Unit("m")

In [3]:
u.s    # The unit of seconds

Unit("s")

In [4]:
u.m / u.s    # combine them into a composite unit

Unit("m / s")

### For any unit you can find all of the built-in units that are equivalent:

In [5]:
u.m.find_equivalent_units()

Primary name,Unit definition,Aliases
AU,1.49598e+11 m,"au, astronomical_unit"
Angstrom,1e-10 m,"AA, angstrom"
cm,0.01 m,centimeter
earthRad,6.3781e+06 m,"R_earth, Rearth"
ft,0.3048 m,foot
fur,201.168 m,furlong
inch,0.0254 m,
jupiterRad,7.1492e+07 m,"R_jup, Rjup, R_jupiter, Rjupiter"
lyr,9.46073e+15 m,lightyear
m,irreducible,meter


In [6]:
u.s.find_equivalent_units()

Primary name,Unit definition,Aliases
a,3.15576e+07 s,annum
d,86400 s,day
fortnight,1.2096e+06 s,
h,3600 s,"hour, hr"
min,60 s,minute
s,irreducible,second
sday,86164.1 s,
wk,604800 s,week
yr,3.15576e+07 s,year


### The `units` package is much more useful when you combine it with scalars or arrays to create `Quantities`

In [7]:
position = np.array([2,3,5,7,11,13])

position

array([ 2,  3,  5,  7, 11, 13])

In [8]:
position = np.array([2,3,5,7,11,13]) * u.m

position

<Quantity [ 2.,  3.,  5.,  7., 11., 13.] m>

In [9]:
my_time = 0.25 * u.s

velocity = position / my_time

velocity

<Quantity [ 8., 12., 20., 28., 44., 52.] m / s>

### You can access the number and unit part of the Quantity separately:

In [10]:
velocity.value

array([ 8., 12., 20., 28., 44., 52.])

In [11]:
velocity.unit

Unit("m / s")

### This is useful in formatting output:

In [12]:
f"The velocity of the first particle is {velocity[0].value:.1f} in the units of {velocity[0].unit:s}."

'The velocity of the first particle is 8.0 in the units of m / s.'

#### At example problem: How long would it take to go 100 km at velocities in `velocity`?

In [13]:
my_distance = 100 * u.km

mt_other_time = my_distance / velocity

mt_other_time

<Quantity [12.5       ,  8.33333333,  5.        ,  3.57142857,  2.27272727,
            1.92307692] km s / m>

#### Notice that the units are a bit strange. We can simplify this using `.decompose()`

In [14]:
mt_other_time.decompose()

<Quantity [12500.        ,  8333.33333333,  5000.        ,  3571.42857143,
            2272.72727273,  1923.07692308] s>

#### You do not have to worry about working in different units (**as long as they are the same type**)!

In [16]:
velocity_one = 10.0 * u.m /u.s
velocity_two = 10.0 * imperial.mi / u.h

velocity_one, velocity_two

(<Quantity 10. m / s>, <Quantity 10. mi / h>)

##### ...Notice the difference when using imperial units

In [17]:
velocity_one_two = velocity_one + velocity_two

velocity_one_two

<Quantity 14.4704 m / s>

### Unit conversion is really easy!

In [18]:
velocity_one_two.to(u.cm / u.h)

<Quantity 5209344. cm / h>

In [19]:
velocity_one_two.to(imperial.mi / u.h)

<Quantity 32.36936292 mi / h>

In [20]:
velocity_one_two.si                     # quick conversion to SI units

<Quantity 14.4704 m / s>

In [21]:
velocity_one_two.cgs                    # quick conversion to CGS units

<Quantity 1447.04 cm / s>

### Units and Functions

In [22]:
my_velocity = [1,2,3,4] * imperial.mi / u.day
my_distance = [4,3,2,1] * (1000 * u.m)

---

##### Careful with `()`

In [23]:
[4,3,2,1] * (1000 * u.m)

<Quantity [4000., 3000., 2000., 1000.] m>

In [24]:
[4,3,2,1] * 1000 * u.m

<Quantity [4., 3., 2., ..., 3., 2., 1.] m>

---

In [25]:
def find_time(velocity, distance):

    result = distance / velocity
    return result.decompose()

In [26]:
find_time(my_velocity, my_distance)

<Quantity [214745.88403722,  80529.70651396,  35790.98067287,
            13421.61775233] s>

### Be careful adding units to something that already has units!

In [27]:
def find_time_wrong(velocity, distance):

    result = distance / velocity
    return result * u.day

In [28]:
find_time_wrong(my_velocity, my_distance).decompose()

<Quantity [1.85540444e+10, 6.95776664e+09, 3.09234073e+09, 1.15962777e+09] s2>

In [29]:
def find_time_day(velocity, distance):

    result = distance / velocity
    return result.to(u.day)

In [30]:
find_time_day(my_velocity, my_distance)

<Quantity [2.48548477, 0.93205679, 0.41424746, 0.1553428 ] d>

### Be careful combining quantities with different units!

In [31]:
my_velocity + my_distance

UnitConversionError: Can only apply 'add' function to quantities with compatible dimensions

In [32]:
2 + my_distance

UnitConversionError: Can only apply 'add' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)

### Units make math with Time easy!

In [33]:
day_one = 29 * u.day + 7 * u.h + 56 * u.min + 12 * u.s
day_two = 2 * u.fortnight + 1.33 * u.day

In [34]:
day_one, day_two

(<Quantity 29.33069444 d>, <Quantity 2.095 fortnight>)

In [35]:
day_one - day_two

<Quantity 0.00069444 d>

In [36]:
(day_one - day_two).to(u.min)

<Quantity 1. min>

### You can define your own units

In [39]:
ringo = u.def_unit('Ringos', 3.712 * imperial.yd)

In [40]:
position.to(ringo)

<Quantity [0.5892313 , 0.88384695, 1.47307826, 2.06230956, 3.24077217,
           3.83000347] Ringos>

In [41]:
velocity.to(ringo / u.s)

<Quantity [ 2.35692521,  3.53538782,  5.89231303,  8.24923824, 12.96308867,
           15.32001388] Ringos / s>

##### ...Since `ringo` is self-defined it does not have a `u.` in front of it

### Dimentionless Units

In [37]:
dimless_y = (1 * u.m) / (1 * u.km)

dimless_y

<Quantity 1. m / km>

In [38]:
dimless_y.unit

Unit("m / km")

In [42]:
dimless_y.decompose()   # returns the scale of the dimentionless quanity

<Quantity 0.001>

### Some math functions only make sense with dimentionless quanities

In [43]:
np.log(2 * u.m)

UnitTypeError: Can only apply 'log' function to dimensionless quantities

In [44]:
np.log((2 * u.km) / (1 * u.m))

<Quantity 7.60090246>

In [45]:
np.log10((2 * u.km) / (1 * u.m))

<Quantity 3.30103>

### Or they expect the correct type of unit!

In [46]:
np.sin(2 * u.m)

UnitTypeError: Can only apply 'sin' function to quantities with angle units

In [47]:
np.sin(2 * u.deg)

<Quantity 0.0348995>

## Using units can save you headaches. 

* All of the trig functions expect all angles to be in radians. 
* If you forget this, it can lead to problems that are hard to debug

$$
\sin(90^{\circ}) + \sin(45^{\circ}) =  1 + \frac{\sqrt{2}}{2} \approx 1.7071
$$

In [48]:
np.sin(90) + np.sin(45)

1.7449001881346762

In [49]:
np.sin(90 * u.deg) + np.sin(45 * u.deg)

<Quantity 1.70710678>

---

# Constants

The `Astropy` package also includes a whole bunch of built-in constants to make your life easier.

### [Astropy Constants](http://docs.astropy.org/en/stable/constants/index.html#reference-api)

In [50]:
const.G

<<class 'astropy.constants.codata2018.CODATA2018'> name='Gravitational constant' value=6.6743e-11 uncertainty=1.5e-15 unit='m3 / (kg s2)' reference='CODATA 2018'>

In [51]:
const.M_sun

<<class 'astropy.constants.iau2015.IAU2015'> name='Solar mass' value=1.988409870698051e+30 uncertainty=4.468805426856864e+25 unit='kg' reference='IAU 2015 Resolution B 3 + CODATA 2018'>

---

### An Example: The velocity of an object in circular orbit around the Sun is

$$\large
v=\sqrt{GM_{\odot}\over d} 
$$

In [52]:
my_distance = 1 * u.AU

In [53]:
def find_orbit_v(distance):
    result = np.sqrt(const.G * const.M_sun / distance)
    return result.decompose()

In [54]:
orbit_v = find_orbit_v(my_distance)
orbit_v

<Quantity 29784.69182968 m / s>

In [55]:
orbit_v.to(u.km/u.s)

<Quantity 29.78469183 km / s>

In [56]:
orbit_v.to(ringo/u.ms)

<Quantity 8.77503639 Ringos / ms>

#### Be careful about the difference between a unit and a constant

In [57]:
my_star = 1 * u.solMass
my_star

<Quantity 1. solMass>

In [58]:
my_star.unit

Unit("solMass")

In [59]:
const.M_sun

<<class 'astropy.constants.iau2015.IAU2015'> name='Solar mass' value=1.988409870698051e+30 uncertainty=4.468805426856864e+25 unit='kg' reference='IAU 2015 Resolution B 3 + CODATA 2018'>

In [60]:
const.M_sun.unit

Unit("kg")

## Last week's homework

In [61]:
def find_diameter(ab_mag, albedo):
    result = (1329 / np.sqrt(albedo)) * (10 ** (-0.2 * ab_mag))
    return result * u.km

In [62]:
my_ab_mag = 3.34
my_albedo = 0.09

asteroid_diameter = find_diameter(my_ab_mag, my_albedo)
asteroid_diameter

<Quantity 951.48890004 km>

In [63]:
def find_mass(diameter, density):
    result = density * (1/6) * np.pi * diameter ** 3
    return result.decompose()

In [64]:
my_density = 3000 * (u.kg / u.m**3)

asteroid_mass = find_mass(asteroid_diameter, my_density)

In [65]:
asteroid_mass

<Quantity 1.35310362e+21 kg>

In [66]:
moon_mass = u.def_unit('Lunar_Masses', 7.34767309e22 * u.kg)

In [67]:
asteroid_mass.to(moon_mass)

<Quantity 0.0184154 Lunar_Masses>